In [20]:
import json
import csv
import pandas as pd
import numpy as np

In [21]:
df_iofrol = pd.read_csv('iofrol.csv', sep=';',)
df_iofrol.shape

(32260, 7)

In [22]:
df_iofrol.head()

,Id,Name,Duration,LastRun,LastResults,Verdict,Cycle
0,1,78082,57262,2015-02-13 16:13:00,[],1,1
1,2,78083,57145,2015-02-13 16:13:00,[],1,1
2,3,78084,56861,2015-02-13 16:13:00,[],1,1
3,4,78085,55332,2015-02-13 16:13:00,[],1,1
4,5,78086,59120,2015-02-13 16:13:00,[],1,1


In [23]:
df_iofrol['Name'].nunique()


1941

In [24]:
# Dropping rows containing LastResults less than size of 3 executions
MF = df_iofrol.loc[:,'LastResults'].tolist()                                                           


i=0
for a in MF:
    
    
    
    s=''.join(x for x in a if x.isdigit())
    if len(s) < 3:
        
        df_iofrol.drop(i , inplace=True)
        
        
    i=i+1


In [25]:
# Based on RETECS documentation for comparison
#df_iofrol=df_iofrol[:2086]

In [26]:
# Adding List values of LastResults into separate E1,E2,E3 columns        
MF = df_iofrol.loc[:,'LastResults'].tolist()                                                           

rows = len(MF)

E1 = []
E2 = []

E3 = []
 
    
i=0
for a in MF:
    
    
    
    s=''.join(x for x in a if x.isdigit())

        

    E1.append(s[0])
    E2.append(s[1])
    E3.append(s[2])
    #print (s[0],"-",s[1],"-",s[2])

    
    
    
    
    
    i=i+1
    #print ("---")

In [27]:
df_iofrol['E1'] = E1
df_iofrol['E2'] = E2
df_iofrol['E3'] = E3


In [28]:
#function to reverse each row of the matrix
def reverse_row_element(array_data):
    list_data = array_data.tolist()
    list_data.reverse()
    return list_data
df_iofrol['E1'] =df_iofrol['E1'].astype(int)
df_iofrol['E2'] =df_iofrol['E2'].astype(int)
df_iofrol['E3'] =df_iofrol['E3'].astype(int)

# defining failure matrix MF
MF = df_iofrol.loc[:,'E1':'E3'].values

rows = len(MF)
columns = len(MF[0])

#calculating distance of failure for each test cases
distances_list = []

for i in range(rows):
    reversed_element = reverse_row_element(MF[i])
    if -1 in reversed_element:        
        for j in range(len(reversed_element)):
            if reversed_element[j] == -1:
                distances_list.insert(i, j+1)
                break
    else:
        #If distance == 0, means the test case has always been successful
        distances_list.insert(i, 0)

In [29]:
#calculating number of change of state of Pass/Fail for each test cases
change_in_status = []

for i in range(rows):
    each_element = MF[i]
    counter = 0
    for j in range(len(each_element)-1):
        if each_element[j]!=each_element[j+1]:
            counter+=1
    change_in_status.append(counter)

In [30]:
#Preparing weights based on DeepOrder algorithm
weights = np.array([0.1,0.2,0.7])
print (weights.shape)
#Extracting average execution time from dataset
df_iofrol['Duration'] =df_iofrol['Duration']

Te = df_iofrol.Duration.values
print (MF.shape)
#Calculating priority value for each test case
priority_value = (MF*weights).sum(axis=1)
priority_value+= np.max(Te)/Te

(3,)
(26599, 3)


In [31]:
df_iofrol['DIST']=distances_list
df_iofrol['CHANGE_IN_STATUS']=change_in_status
df_iofrol['PRIORITY_VALUE'] = priority_value
#df_iofrol.to_csv('iofrol_updated'+'.csv', index = False)

In [32]:
# Update dates in LastRun according to each cycle
from datetime import timedelta


from datetime import datetime
Month_of_thirty_days = [2,4,6,9,11]
from dateutil.relativedelta import relativedelta

currentDay = datetime.now().day
currentMonth = datetime.now().month
currentYear = datetime.now().year


cycle = df_iofrol.loc[:,'Cycle'].values
Dates = df_iofrol.loc[:,'LastRun'].values

previous = 0
rows = len(cycle)
count=0
firsttime = False


# for rest of the cycle
for i in range(rows):
    each_element =cycle[i]
    #print ("Cycle :",each_element)
    #print (Dates[i])
    if i==0:
        #print ("nothing")
        datestring = Dates[i]
        dt = datetime.strptime(datestring, '%Y-%m-%d %H:%M:%S')
        Dates[i] =  str(dt.year) +"-"+ str(dt.month) +"-"+ str(dt.day)
    if i >= 1:
        datestring = Dates[i]
        dt = datetime.strptime(datestring, '%Y-%m-%d %H:%M:%S')
        value = float(each_element-1)
        # print (timedelta(days=value))
        dtt =  dt + timedelta(days=value)
        Dates[i] =  str(dtt.year) +"-"+ str(dtt.month) +"-"+ str(dtt.day)
    #print (Dates[i])
    #print ('---')  
    



 

In [33]:
#for testing purposes only
from datetime import datetime
datestring = Dates[0]
dt = datetime.strptime(datestring, '%Y-%m-%d')
#print (dt.year, dt.month, dt.day)

from datetime import date
f_date = date(dt.year, dt.month, dt.day)
l_date = date(currentYear,currentMonth,currentDay)
delta = l_date - f_date
print(delta.days)
days.append(delta.days)

2062


In [34]:
# Calculating difference in LastRun test cases (number of days in return)
from datetime import datetime
Month_of_thirty_days = [2,4,6,9,11]
from dateutil.relativedelta import relativedelta

currentDay = datetime.now().day
currentMonth = datetime.now().month
currentYear = datetime.now().year


days=[]
previous = 0
rows = len(Dates)
count=0
firsttime = False
'''
#for 1st Cycle
from datetime import datetime
datestring = Dates[0]
dt = datetime.strptime(datestring, '%Y-%m-%d')
#print (dt.year, dt.month, dt.day)

from datetime import date
f_date = date(dt.year, dt.month, dt.day)
l_date = date(currentYear,currentMonth,currentDay)
delta = l_date - f_date
print(delta.days)
days.append(delta.days)
'''



# for rest of the cycle
for i in range(rows):
    each_element =cycle[i]
    #print ("Cycle :",each_element)
    
    
    
    if i ==0:
        #print ("1st Cycle")
        #for 1st Cycle
        from datetime import datetime
        datestring = Dates[i]

        dt = datetime.strptime(datestring, '%Y-%m-%d')
        #print (type(dt))
    
        f_date = date(dt.year, dt.month, dt.day)
        l_date = date(currentYear,currentMonth,currentDay)
        delta = l_date - f_date
        #print (delta)
        #print (currentYear, currentMonth, currentDay, " - ", dt.year, dt.month, dt.day)

        previous = each_element
        
    if i >= 1:
        datestring = Dates[i]
        datestring_prev = Dates[i-1]
        dt = datetime.strptime(datestring, '%Y-%m-%d')
        dt_prev = datetime.strptime(datestring_prev, '%Y-%m-%d')


        f_date = date(dt.year, dt.month, dt.day)
        l_date = date(dt_prev.year, dt_prev.month, dt_prev.day)
        delta = l_date - f_date
        #print ("** ",dt_prev.year, dt_prev.month, dt_prev.day, " - ", dt.year, dt.month, dt.day)
        

    days.append(delta.days)
    
    #print ('---')  
            
    
    
    

In [35]:
df_iofrol['LastRunFeature'] = days


In [36]:
df_iofrol["LastRunFeature"]=((df_iofrol["LastRunFeature"]-df_iofrol["LastRunFeature"].min())/(df_iofrol["LastRunFeature"].max()-df_iofrol["LastRunFeature"].min()))*5
df_iofrol["DurationFeature"]=((df_iofrol["Duration"]-df_iofrol["Duration"].min())/(df_iofrol["Duration"].max()-df_iofrol["Duration"].min()))*5
df_iofrol["Duration"] = df_iofrol["Duration"].astype(int)


In [37]:
df_iofrol['Id'] = range(1, 1+len(df_iofrol))

In [38]:
df_iofrol.to_csv('iofrol_updated'+'.csv', index = False)

In [ ]:
df_iofrol.head()